In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from patsy import dmatrices

In [6]:
df = pd.read_excel('POSE-205.xlsx')

In [7]:
# vars = ['name', 'humidity_avg', 'pm10_avg', 'pm25_avg', 'pressure_avg', 'temperature_avg']
vars = ['name', 'city']
df2 = df[vars]
df2.dropna(inplace=True)
df2.drop_duplicates(inplace=True)
df2.reset_index(drop=True, inplace=True)
print(df2)



# df2 = df[vars]

# df2[-5:]

                                                   name                  city
0     SZKOŁA PODSTAWOWA NR 8 IM. MARTYROLOGII PIAŚNI...             WEJHEROWO
1                       SZKOŁA PODSTAWOWA NR 48 W GDYNI                GDYNIA
2     SZKOŁA PODSTAWOWA IM. ARKADEGO FIEDLERA W PRZY...            PRZYBOROWO
3     ZESPÓŁ SZKÓŁ IM. KONSTYTUCJI 3 MAJA W POBIEDZI...  POBIEDZISKA LETNISKO
4                  ZESPÓŁ SZKOLNO-PRZEDSZKOLNY W MODRZU                MODRZE
...                                                 ...                   ...
1770  Technikum nr 2 w Zespole Szkół Budowlanych w B...           Bolesławiec
1771  Zespół Szkół Ogólnokształcących i Zawodowych i...           Bolesławiec
1772  Technikum Informatyczne przy Noworudzkiej Szko...             Nowa Ruda
1773                SPOŁECZNA SZKOŁA PODSTAWOWA W GZACH                   GZY
1774  SZKOŁA PODSTAWOWA IM. PPOR. EMILII GIERCZAK W ...                  ŁĄCK

[1775 rows x 2 columns]


C:\Users\qucker135\AppData\Local\Temp\ipykernel_13116\305767049.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.dropna(inplace=True)
C:\Users\qucker135\AppData\Local\Temp\ipykernel_13116\305767049.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.drop_duplicates(inplace=True)


In [18]:
import spacy

nlp = spacy.load("pl_core_news_sm")
doc = nlp("Nazywam się Cezary Baryka, i od 2 tygodni jestem właścicielem tego szklanego domu. Hej dzieci, jestem Papa Słoń co robicie tak daleko od miasta?")

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Cezary Baryka 12 25 persName


In [35]:
# y, X = dmatrices('pressure_avg ~ humidity_avg + pm10_avg', data=df2, return_type='dataframe')
import spacy
from collections import Counter

nlp = spacy.load("pl_core_news_lg")
# lemmatizer = nlp.get_pipe("lemmatizer")
# print(lemmatizer.mode)

doc = nlp(' | '.join(df2['name']).title())

person_lemmas = []

for ent in doc.ents:
    if ent.label_ == 'persName':
        # print(ent.text, ent.start_char, ent.end_char, ent.label_)
        # l.append(' '.join([token.lemma_ for token in ent]))
        person_lemmas.append(ent.lemma_)

# convert to dictionary with number of occurences

# d = dict(Counter(l))
# print(d)
person_lemmas = [x.title() for x in person_lemmas if len(x) > 3]

# for each person entity, get the basic form of the name

# get the most common names
most_common = dict(Counter(person_lemmas).most_common(10))
if 'Jan Paweł Ii W' in most_common:
    most_common['Jan Paweł II'] = most_common['Jan Paweł Ii W']
    most_common.pop('Jan Paweł Ii W')
most_common = dict(sorted(most_common.items(), key=lambda item: item[1], reverse=True))
print(most_common)

{'Jan Paweł II': 56, 'Mikołaj Kopernik': 45, 'Maria Konopnicka': 42, 'Adam Mickiewicz': 19, 'Janusz Korczak': 16, 'Jan Kochanowski': 16, 'Jan Twardowski': 14, 'Kornel Makuszyński': 14, 'Stefan Wyszyński': 13, 'Henryk Sienkiewicz': 13}


In [1]:
import spacy

nlp = spacy.load("en_core_web_sm")
print("Pipeline:", nlp.pipe_names)
doc = nlp("I was reading the paper.")
token = doc[0]  # 'I'
print(token.morph)  # 'Case=Nom|Number=Sing|Person=1|PronType=Prs'
print(token.morph.get("PronType")) 

Pipeline: ['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']
Case=Nom|Number=Sing|Person=1|PronType=Prs
['Prs']


In [26]:
import spacy

# Załadowanie modelu językowego
nlp = spacy.load('pl_core_news_sm')

# Przykładowy tekst
text = "Lech Wałęsa urodził się w Polsce i jest byłym prezydentem. Był świadkiem wyboru Jana Pawła Ii na papieża oraz ustanowienia Marii Konopnickiej sekretarzem stanu."

# Przetworzenie tekstu
doc = nlp(text)

# Wyodrębnienie nazw własnych (named entities) i ich podstawowych form
entities = [(ent.text, ent.lemma_, ent.label_) for ent in doc.ents]

# Wyświetlenie wyników
for entity in entities:
    print(f"Tekst: {entity[0]}, Podstawowa forma: {entity[1]}, Typ: {entity[2]}")

Tekst: Lech Wałęsa, Podstawowa forma: Lech Wałęsa, Typ: persName
Tekst: Polsce, Podstawowa forma: Polska, Typ: placeName
Tekst: Jana Pawła Ii, Podstawowa forma: Jan Paweł Ii, Typ: persName
Tekst: Marii Konopnickiej, Podstawowa forma: Maria Konopnicka, Typ: persName
